In [34]:
import pandas as pd
from selenium import webdriver # Navegador
from selenium.webdriver.common.by import By #Para seleccionar elementos html
from selenium.webdriver.common.keys import Keys # Para manejar entrada de teclada
from selenium.common.exceptions import NoSuchElementException # En caso de que no se encuentre el elemento.
import time #Libreria para que el programa quede en stand by durante algunos segundos

In [35]:
# Definimos el navegador a usar
driver = webdriver.Chrome(executable_path= 'src/chromedriver.exe')

#Definimos la url a trabajar
url = 'https://www.imdb.com/'

C:\Users\esequ\AppData\Local\Temp\ipykernel_5856\1274925799.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= 'src/chromedriver.exe')


In [36]:
try:  # Ejecutamos:
    driver.get(url)  # Redireccionamos el navegador
    # Si el texto se encuentra en el codigo, sigue la ejecucion
    assert 'IMDb: Puntuaciones, reseñas y dónde ver las mejores películas y programas de televisión' in driver.title #Driver.title hace referencia al tag title en el html
except AssertionError:  # Si obtenemos un error se imprime un msj de error
    print('No se ah podido conectar con la pagina')


In [37]:
# Buscamos el botón menú.
elemento = driver.find_element(By.ID, 'imdbHeader-navDrawerOpen') # Buscamos un elemnto por ID
# Hacemos click en el elemento.
elemento.click()
time.sleep(2) # esperamos dos segundos para que la pagina cargue bien

In [38]:
# Buscamos el enlace a las peliculas mas populares
elemento = driver.find_element(By.LINK_TEXT, 'Películas más populares') #Obtenemos el elemnto que nos llevara a las peliculas mas populares
#Hacemos click en el elemento
elemento.click()

In [39]:
# Obtener elementos de la tabla de peliculas.

#Obtenemos todos las etiquetas a dentro de la segunda columna de cada fila, usar find_elements con s para obtener mas de un elemento
lista_peliculas = driver.find_elements(By.XPATH, '//*[@id="main"]/div/span/div/div/div[3]/table/tbody/tr/td[2]/a') 

#Creamos una lista para guardar los enlaces
links_peliculas = []

for pelicula in lista_peliculas: #Por cada elemento a dentro de movies_list
    links_peliculas.append(pelicula.get_attribute('href')) #Agregamos a la lista movies_links la url

In [40]:
nombre = [] #Creamos una lista para el nombre de la pelicula
anio = [] #Creamos una lista para el año de la pelicula
rating = [] #Creamos una lista para el rating de la pelicula
sipnosis = [] #Creamos una lista para la sipnosis de la pelicula
generos = [] #Creamos una lista para los generos de la pelicula
director = [] #Lista para el director de la pelicula

for link in links_peliculas:

    driver.get(link)

    #Utilizamos try y except para comprobar si el elemento existe en la pagina
    #Si el elemento existe, se guarda su contenido
    #Si el elemento no existe, se ingresa un campo vacio

    try:
        nombre.append(driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1').text)
    except NoSuchElementException:
        nombre.append('')

    try:
        anio.append(driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[1]/a').text)
    except NoSuchElementException:
        anio.append('')

    try:
        rating.append(driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/div/div/div[2]/div[1]/span[1]').text)
    except NoSuchElementException:
        rating.append('')

    try:
        sipnosis.append(driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/p/span[2]').text)
    except NoSuchElementException:
        sipnosis.append('')

    try:
        #Creamos una lista para todos los generos de la pelicula
        lista_generos = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/div/div[2]/a/span')
        # Por cada elemento en la lista, generamos un indice para el elemento y el susodicho elemento
        for indice, item in enumerate(lista_generos): # Utilizamos enumerate para obtener el indice y el elemento
            lista_generos[indice] = item.text # Ingresamos el texto del elemento en la posicion de memoria donde se encuentra
        # Ingresamos los generos
        generos.append(lista_generos)

    except NoSuchElementException:
        generos.append('')

    try:
        director.append(driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]/div/ul/li/a').text)
    except NoSuchElementException:
        director.append('')

In [41]:
#Creamos el dataframe con la informacion que conseguimos
movies_df = pd.DataFrame({'name': nombre, 'year': anio, 'rating': rating, 'sipnosis': sipnosis, 'generos': generos, 'director': director})

In [42]:
#Exportamos un csv
movies_df.to_csv('data/movie_dataset.csv', index= False)